# CNN Modl
- In this file we can simple Change the model Artitucture rest of the code will same to the previous file.
- Simple Sequentail Model change to CNN Modle.

# Step
- Get data
- Build Model
- Train Model
- Evulate Model
- Save the Model

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader # for data loading
from torchvision import datasets # prebuild datasets
import torchvision
from torchvision.transforms import ToTensor # Transformations

# Load Dataset
- We will use the prebuild dataset from troch so that we can train our custom model.
- We will use the fashion Mnist Dataset which have 10 classes.

In [2]:
# we wil use pre build image net dataset
train_data = torchvision.datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

# Test data
test_data = torchvision.datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

100%|██████████| 26.4M/26.4M [00:02<00:00, 11.1MB/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29.5k/29.5k [00:00<00:00, 172kB/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4.42M/4.42M [00:01<00:00, 3.04MB/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5.15k/5.15k [00:00<00:00, 23.8MB/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [3]:
train_data

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [4]:
test_data

Dataset FashionMNIST
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: ToTensor()

# Observation
- we can get the data successfully.
- Now we can make a dataloader so that we can load the data efficently.

In [5]:
# Train Loader
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)

# Test Loader
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

In [6]:
# View the barch
for x, y in train_loader:
    print(x.shape)
    print(y.shape)
    break

torch.Size([32, 1, 28, 28])
torch.Size([32])


# Build Model
- Now we can build a CNN Model with 3 conv layers and 1 fully connected layers

In [7]:
class MyCNN(nn.Module):
    def __init__(self):
        super(MyCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32,kernel_size=3,padding=1,stride=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64,kernel_size=3,padding=1,stride=1)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128,kernel_size=3,padding=1,stride=1)

        self.relu = nn.ReLU()

        # Flatten the layers
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(in_features=128*28*28, out_features=10)


    def forward(self, x):
        x = self.relu(self.conv1(x))

        x = self.relu(self.conv2(x))

        x = self.relu(self.conv3(x))

        x= self.flatten(x)

        x= self.fc(x)
        return x

# Observation
- We can make a simple CNN Model with `3 Conv layer` and `1 linear layer.`
- Our Goal is not make the perfect model.

In [8]:
# Set Optimizer and Loss
model =MyCNN()

loss_fn = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=0.0001)

model, loss_fn, optimizer

(MyCNN(
   (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (relu): ReLU()
   (flatten): Flatten(start_dim=1, end_dim=-1)
   (fc): Linear(in_features=100352, out_features=10, bias=True)
 ),
 CrossEntropyLoss(),
 SGD (
 Parameter Group 0
     dampening: 0
     differentiable: False
     foreach: None
     fused: None
     lr: 0.0001
     maximize: False
     momentum: 0
     nesterov: False
     weight_decay: 0
 ))

# make Training Loop

In [9]:
from tqdm import tqdm
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [10]:
model = model.to(device)
epochs = 5
for epoch in range(epochs):
    train_loss = 0
    num_correct = 0
    num_samples = 0
    for image, label in tqdm(train_loader):
        image = image.to(device)
        label = label.to(device)

        # do forward pass
        output = model(image)

        # Calculate the loss
        loss = loss_fn(output,label)

        # Set the optimizer to zero_grad
        optimizer.zero_grad()

        # do backward
        loss.backward()

        optimizer.step()

        train_loss +=loss.item()

        # calculate the accuracy
        _, prediction = output.max(1)
        num_correct += (prediction == label).sum()
        num_samples += prediction.size(0)

    # calculate the accuracy
    accuracy = num_correct/num_samples
    print(f"Epoch: {epoch + 1}/{epochs}, Train Loss: {train_loss / len(train_loader):.4f}, Train Accuracy: {accuracy * 100:.2f}%")

100%|██████████| 1875/1875 [00:19<00:00, 97.61it/s]


Epoch: 1/5, Train Loss: 2.2593, Train Accuracy: 35.44%


100%|██████████| 1875/1875 [00:16<00:00, 115.50it/s]


Epoch: 2/5, Train Loss: 2.0110, Train Accuracy: 60.66%


100%|██████████| 1875/1875 [00:14<00:00, 126.14it/s]


Epoch: 3/5, Train Loss: 1.2047, Train Accuracy: 68.33%


100%|██████████| 1875/1875 [00:14<00:00, 125.69it/s]


Epoch: 4/5, Train Loss: 0.7828, Train Accuracy: 74.20%


100%|██████████| 1875/1875 [00:15<00:00, 123.78it/s]

Epoch: 5/5, Train Loss: 0.6829, Train Accuracy: 76.21%


In [11]:
128*28*28

100352


# Model Evulation

In [12]:
model.eval()
with torch.no_grad():
    test_loss = 0
    num_correct = 0
    num_samples = 0
    for image, label in tqdm(test_loader):
        image = image.to(device)
        label = label.to(device)

        # do forward pass
        output = model(image)

        # calculate the loss
        loss = loss_fn(output,label)

        test_loss += loss.item()

        # calculate the accuracy
        _,prediction = output.max(1)
        num_correct += (prediction == label).sum()
        num_samples += prediction.size(0)

    accuracy = num_correct/num_samples
    print(f"Test Loss: {test_loss / len(test_loader):.4f}, Test Accuracy: {accuracy * 100:.2f}%")

100%|██████████| 313/313 [00:02<00:00, 154.89it/s]

Test Loss: 0.6736, Test Accuracy: 76.08%


# Observation
- We see that we can train the model in 5 epochs and we will get the 76% accuracy in this dataset with simple CNN model.
- we will definetly improve the model